<a href="https://colab.research.google.com/github/Charanpoojary/Fake-News-Prediction-using-Machine-Learning/blob/main/Fake_News_Prediction_using_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





In [2]:
# importing libraries
import numpy as np    # to create arrays
import pandas as pd    # to create dataframes
import re                       # re= regular expression( used for searching a text in a document )
from nltk.corpus import stopwords       #nltk natural language toolkit   , stop words = which doesnt add value to the paragraph,-which is of no use for ex- where,was, what, the, n all
# by using stopwors we can remove those words,
from nltk.stem.porter import PorterStemmer     # stem= stemming ( removes the prefixes and sufixes of the words)
from sklearn.feature_extraction.text import TfidfVectorizer      # TfidfVectorizer used to convert the text into numbers '
from sklearn.model_selection import train_test_split      # for splitting the data into train and test
from sklearn.linear_model import LogisticRegression       # importing the model that is needed for the operationn
from sklearn.metrics import accuracy_score                # for evaluation

In [3]:
# we are going to download all the stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# now let us print the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
# so we are going to remove these  words from our dataset

In [6]:
# data collection

In [7]:
# mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# reading teh file path
file_path = '/content/drive/MyDrive/Datasets/fakenewstrain.csv'
df=pd.read_csv(file_path)

In [9]:
df.head()   # returns the top 5 rows of the dataset

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
df.shape      # returns the rows and columnns of the dataset

(20800, 5)

In [11]:
df.info()      # returns the information about the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [12]:
df.describe()    # returns the information about numerical columns

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [13]:
df.isnull().sum()  #checking for the null values

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [14]:
# here we can replace the null values with the empty string

In [15]:
df=df.fillna('')      # replace the null values with empty string

In [16]:
# we have total 5 columns now we are going to merge author and title columns

In [17]:
df['content']=df['author']+' '+df['title']   # creating a new column

In [18]:
print(df['content'])   # printing the new column

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [19]:
df.head()   # returns the top 5 rows of the dataset

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [20]:
#separatiing the data and label

In [21]:
X= df.drop('label',axis=1)   # independant variable
Y= df['label']               # dependant variable

In [22]:
# printing independant variable & dependant variable
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [23]:
port_stem=PorterStemmer()   # initiating stemmer

In [24]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)   # going to remove everything except numbers
    stemmed_content = stemmed_content.lower()           # converting every word into lower case
    stemmed_content = stemmed_content.split()           # splitting the words
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]    #  taking the words which are not there in stopwords and applying portstem
    stemmed_content = ' '.join(stemmed_content)         # at the end joining again
    return stemmed_content

In [26]:
df['content']=df['content'].apply(stemming) # applying the function to content column

In [28]:
print(df['content'])    # printing the column after applying the stemming function

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [30]:
# now again we will separate the dataset ( because now we have stemmed data which is faster compare to the original dataset)

In [29]:
#separating the data and label   ( we are taking only 2 tables now)
X = df['content'].values        # independant variables
Y = df['label'].values          # dependant variables

In [32]:
# printing X and Y
print(X)   
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [33]:
# now we have to convert all the input text into numerical data

In [35]:
text_numeric=TfidfVectorizer()   #intiating TfidfVectorizer
text_numeric.fit(X)              # fitiing the data for conversion 
X=text_numeric.transform(X)      # conversion

In [36]:
print(X)         # printing the converted data

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [37]:
# now splitting the dataset into train and test split
X_train, X_test, Y_train, Y_test=train_test_split(X, Y, test_size=0.2, stratify= Y, random_state=2)

In [39]:
# printing the shape for original and splitted data
print(X.shape, X_train.shape, X_test.shape)

(20800, 17128) (16640, 17128) (4160, 17128)


In [41]:
# training the model and predicting    ( logistic regression)

In [40]:
model = LogisticRegression() # initiating the model
model.fit(X_train,Y_train)   # fitting the data for training

LogisticRegression()

In [49]:
# prediction on trained data

In [44]:
Y_train_pred=model.predict(X_train)

In [ ]:
# evaluation 

In [46]:
training_accuracy=accuracy_score(Y_train,Y_train_pred)

In [52]:
print(training_accuracy)  # printing the training accuracy

0.9865985576923076


In [54]:
# prediction on test data

In [55]:
Y_test_pred=model.predict(X_test)

In [57]:
# evaluation 

In [58]:
test_accuracy=accuracy_score(Y_test,Y_test_pred)

In [59]:
print(test_accuracy)  # printing the test accuracy

0.9790865384615385


In [60]:
# we have got 98% accuary while training and 97% in test , so we can say the our model is generalized

In [61]:
# now lets check our model on a new data

In [65]:
X_new=X_test[0]   # taking the first value of the test data
prediction=model.predict(X_new)  # model prediction on new data
print(prediction)     # printing prediction

[1]


In [67]:
if prediction[0]==1:
  print('The news is fake')
else:
  print('The news is real')

The news is fake


In [ ]:
# lets check the above prediction

In [68]:
print(Y_test[0])    # because we have the answer in Y_test

1


In [ ]:
# lets take another example

In [70]:
X_new=X_test[1]   # taking the 2nd value of the test data
prediction=model.predict(X_new)  # model prediction on new data
print(prediction)     # printing prediction

[0]


In [71]:
if prediction[0]==1:
  print('The news is fake')
else:
  print('The news is real')

The news is real


In [ ]:
# lets check the above prediction

In [72]:
print(Y_test[1])    # because we have the answer in Y_test

0


In [73]:
# our model is predicting in a better way